# TextGen.ipynb
Text generation example<br>
COSC 480 - Deep Learning<br>
Fall 2018<br>
Alan C. Jamieson<br>
Last updated: 10/8/18<br>

Minor modifications from source: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

For this example, we'll pull a text file with representative text written by Edgar Allan Poe
and do a really bad job of generating text that looks like Edgar Allan Poe's work (sorry, Edgar).
This is, of course, on theme since it is close to Halloween, and close to his final resting place
in Baltimore.

In [31]:
# imports needed
import numpy
import sys
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [32]:
#load our file and convert to a consistent case
#make sure file is in the same directory as the notebook
filename = "Desktop\\short.txt"
#filename = "spaceCleaned.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [36]:
#map our chars to integers so that we can use them properly
#chars = sorted(list(set(raw_text)))
#char_to_int = dict((c, i) for i, c in enumerate(chars))
t = Tokenizer(filters = '')
t.fit_on_texts([raw_text])
encoded_docs = t.texts_to_sequences([raw_text])[0]
print(t.word_index)

{'can': 76, 'oh-hoh': 183, 'all': 10, 'swim?': 256, 'here?': 159, 'across': 161, 'see': 120, 'likes': 312, 'fodder': 192, 'imagine': 77, 'now': 22, 'couple': 124, 'design': 194, 'character': 58, "that's": 35, 'motivations,': 197, 'trap': 279, 'whether': 200, 'over': 326, 'alright,': 252, 'before': 233, 'he': 72, 'themselves"': 300, 'me': 236, 'should': 154, 'a': 2, 'gotten': 140, 'grind': 191, 'way,': 134, 'walking': 334, 'unexciting.': 90, 'take': 116, 'because': 258, 'it': 171, 'out': 25, 'ever': 139, 'mom': 190, 'walls.': 335, 'pool': 253, 'talk': 135, 'font.': 282, '[singing]': 97, 'partner': 275, 'sans"': 218, 'world': 19, 'skeleton': 215, 'references': 220, 'style...': 182, 'ridiculous': 331, 'undertale,': 60, 'is...': 228, 'you': 5, 'sans.': 214, 'for': 26, 'sidekick,': 308, 'me,': 210, 'which': 82, 'regret': 250, 'honestly,': 168, 'warning.': 239, 'also': 69, 'guy.': 232, 'every': 57, 'on': 163, 'kinda': 306, 'feeling': 259, 'brothers': 273, 'were': 113, 'origins': 284, 'battle

In [45]:
#split our text into our X and Y vectors
n_chars = len(encoded_docs)
n_vocab = len(t.word_index)
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = encoded_docs[i:i + seq_length]
  seq_out = encoded_docs[i + seq_length]
  dataX.append([char for char in seq_in])
  dataY.append(seq_out)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  548


In [46]:
#work with the resulting data to make sure that it's in a form that keras will take
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

In [47]:
#create the model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#if you run into issues where the model fails to finish or flat-out crashes the kernel, you may want
#to consider checkpoints and uncomment below, swapping the fit call:
#------uncomment here for checkpoints start
#filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]
#model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)
#------end
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
548/548 [==============================] - 7s 12ms/step - loss: 5.8157
Epoch 2/20
548/548 [==============================] - 2s 3ms/step - loss: 5.7904
Epoch 3/20
548/548 [==============================] - 2s 3ms/step - loss: 5.7170
Epoch 4/20
548/548 [==============================] - 2s 3ms/step - loss: 5.5168
Epoch 5/20
548/548 [==============================] - 2s 4ms/step - loss: 5.4602
Epoch 6/20
548/548 [==============================] - 2s 4ms/step - loss: 5.4328
Epoch 7/20
548/548 [==============================] - 2s 4ms/step - loss: 5.4153
Epoch 8/20
548/548 [==============================] - 2s 4ms/step - loss: 5.4224
Epoch 9/20
548/548 [==============================] - 2s 4ms/step - loss: 5.4055
Epoch 10/20
548/548 [==============================] - 2s 4ms/step - loss: 5.3974
Epoch 11/20
548/548 [==============================] - 2s 4ms/step - loss: 5.3942
Epoch 12/20
548/548 [==============================] - 2s 4ms/step - loss: 5.3863
Epoch 13/20
548/548 [===

In [49]:
#create our prediction
#------uncomment here for checkpoints start
#filename = "yoursmallestlostweightfilehere"
#model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')
#------end
int_to_word = dict((i, c) for i, c in t.word)
oot = ""
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print(pattern)
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate WORDS
for i in range(50):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  #print(result)
  oot = oot + result
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")
print(oot)

[138, 139, 140, 45, 141, 142, 4, 143, 2, 46]
Seed:


KeyError: 138